In [1]:
import pandas as pd

<h4> Import data:

In [3]:
zz = pd.read_csv('../CSV_files/中转.csv',header = None)
zz = zz.rename(columns = {0:'reviews'})
zz.head()

,reviews
0,机场升级，服务降级。今天到T2找了半天中转休息室，走到三楼服务员说我的计分不够，只能提供金银...
1,今天来广州T2中转，服务员说现在不提供中转休息。我6个多小时。严重抗议！我们的权益被南航剥夺
2,页面字体可以稍微大一些。比如中转部分眼神儿不好，根本看不清楚。还有各项功能，也可以稍微大一些...
3,我今天乘坐南航从上海到旧金山的飞机CZ3544，需要在武汉中转。票号7821127333。上...
4,我从始至终选乘南方航空的飞机无论国内行程国外行程 无论票价高低 但是在累积里程的时候我发现 ...


<h4> 分词：

In [6]:
review_0 = zz.reviews.iloc[0]

In [7]:
import jieba
import jieba.posseg as pseg
import jieba.analyse

seg_list = jieba.cut(review_0, cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_d/b3chzbkx5vgg1wtjm942qj4m0000gn/T/jieba.cache
Loading model cost 1.058 seconds.
Prefix dict has been built succesfully.


Default Mode: 机场/ 升级/ ，/ 服务/ 降级/ 。/ 今天/ 到/ T2/ 找/ 了/ 半天/ 中转/ 休息室/ ，/ 走到/ 三楼/ 服务员/ 说/ 我/ 的/ 计分/ 不够/ ，/ 只能/ 提供/ 金银/ 卡/ 会员/ 休息/ 。/ 然后/ 到/ 中转/ 服务/ 询问/ ，/ 回答/ 我/ 南航/ 转/ 到/ 丅/ 2/ 后/ 暂时/ 没有/ 这个/ 服务/ 。/ 想/ 问/ 一下/ 你们/ 可以/ 随便/ 取消/ 这项/ 服务/ 吗/ ？/ 是不是/ 机场/ 再/ 升级/ 一次/ ，/ 下次/ 就/ 取消/ 所有/ 的/ 座位/ 。


In [8]:
# 关键词提取
keywords_0 = jieba.analyse.extract_tags(review_0, topK=20, withWeight=True, allowPOS=('n','nr','ns'))
keywords_0

[('机场', 2.014140778422857),
 ('计分', 1.5508793163285712),
 ('降级', 1.3671349016699998),
 ('服务员', 1.2728921521714285),
 ('座位', 1.1338711869085716),
 ('会员', 1.0640898139342858)]

In [11]:
# 移除stop words 
# 直接采用open file as read 模式；区别于之前使用pandas来process.

# 创建停用词list  
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  

# 对句子进行分词  
def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence.strip())  
    stopwords = stopwordslist('../stopwords.txt')  # 这里加载停用词的路径  
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr 

# handle tf-idf (Q: tf-idf based on single doc. rather than the whole corpus? 有意义吗？)
def get_topN_tf_idf(content,topK=20):
    tags = jieba.analyse.extract_tags(content, topK)
    print(" ".join(tags))
    
inputs = open('../CSV_files/中转.csv', 'r', encoding='utf-8')  
outputs = open('../CSV_files/中转output.txt', 'w')  
for line in inputs:  
    line_seg = seg_sentence(line)  # 这里的返回值是字符串  
#     line_seg = get_topN_tf_idf（line_seg,topK=20)   # 移除低频词
    outputs.write(line_seg + '\n')  
outputs.close()  
inputs.close() 

In [23]:
def get_output_file(inputs_path,outputs_path):
    '''
    This function inputs the source data files, 
    and return the 分词 file as ouput.    
    '''
    
    def stopwordslist(filepath):  
        stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
        return stopwords  

    # 对句子进行分词  
    def seg_sentence(sentence):  
        sentence_seged = jieba.cut(sentence.strip())  
        stopwords = stopwordslist('./stopwords.txt')  # 这里加载停用词的路径  
        outstr = ''  
        for word in sentence_seged:  
            if word not in stopwords:  
                if word != '\t':  
                    outstr += word  
                    outstr += " "  
        return outstr 

    # handle tf-idf (Q: tf-idf based on single doc. rather than the whole corpus? 有意义吗？)
    def get_topN_tf_idf(content,topK=20):
        tags = jieba.analyse.extract_tags(content, topK)
        print(" ".join(tags))
    
    inputs = open(inputs_path, 'r', encoding='utf-8')  
    outputs = open(outputs_path, 'w')  
    for line in inputs:  
        line_seg = seg_sentence(line)  # 这里的返回值是字符串  
#         line_seg = get_topN_tf_idf（line_seg,topK=20)  # 移除低频词
        outputs.write(line_seg + '\n')  
    outputs.close()  
    inputs.close() 


categories = ['中转','出发','到达','售后','性能','机上','行程管理','计划','设计','预订']
    
for cat in categories:
    input_path = '../CSV_files/' + cat + '.csv'
    output_path = '../output_data/' + cat + 'output.txt'
    get_output_file(input_path, output_path)

TODO: word2vec (法1)
TODO: doc2vec  (法2)
TODO: LDA   (法3)

<h4> Data preprocessing:

In [38]:
# TODO: get all labeled data
import glob

'''
combine dataset (multiple categories) into one single category;
add a column called 'label'
'''

files= glob.glob('../output_data/*.txt')

df_lst = []
for f in files:
    label = f.split('/')[-1][:2]
    df = pd.read_csv(f,header=None)
    df['label'] = label
    df_lst.append(df)

all_df = pd.concat(df_lst)
print('the whole dataset include %d reviews'%len(all_df))
all_df = all_df.rename(columns = {0:'review_tokens'})
all_df.head(10)

the whole dataset include 1623 reviews


,review_tokens,label
0,11 月 15 日 提前 预订 2018 年 11 月 27 日 长沙 飞往 沈阳 cz3...,出发
1,航班 延误 登机口 升舱 活动 以原 航班 起飞时间 为准 办理 理解,出发
2,重庆 乌鲁木齐 南航 航班 天气 原因 延误 和田 乘坐 天津 航班,出发
3,沿途 停靠 理解 延误 小时,出发
4,飞机 无故 延误 小时 脸,出发
5,延误 五个 小时 算上 值机 时间 机场 八个 小时 早上 晚上 解释 解决方案 机长 人影...,出发
6,cz3842 航班 延误 投诉无门 十点 五十 起飞 下午 三点 弄 飞机 两个 小时 告知...,出发
7,南航 航班 延误 发 短信 太 严谨 回复 改 航班 用户名 密码 我要 变更 航班 做 延...,出发
8,行李 延误 重大损失,出发
9,确认 航班 延误 订 票 显示 确认,出发


<h4> Build classifier pipeline:

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

def build_clf_pipeline(data, pipeline_lst):
    '''
    This function takes in the whole dataset, and the pipeline set up as a list.
    Output the accuracy of the classifier pipeline.
    '''
    # vectorize data
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(all_df.review_tokens)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
    # build clf
    text_clf_svm = Pipeline(pipeline_lst)
    text_clf_svm.fit(all_df.review_tokens.values, all_df.label.values)
    
    # get accuracy score
    predicted_labels = text_clf_svm.predict(all_df.review_tokens.values)
    accuracy = accuracy_score(all_df.label.values, predicted_labels)
    print('Classifier reaches accuracy score: {:.2f}'.format(accuracy))

    
# SVM clf
pipeline_lst = [
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf-svm',SGDClassifier(penalty='l2',alpha =1e-3, n_iter = 5, random_state = 0))]

SVM_accuracy = build_clf_pipeline(all_df,pipeline_lst)

Classifier reaches accuracy score: 0.83


/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [56]:
# linearSVC
from sklearn.svm import SVC
pipeline_lst = [
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf-linearSVC',SVC(kernel='linear'))]

linearSVC_accuracy = build_clf_pipeline(all_df,pipeline_lst)

Classifier reaches accuracy score: 0.83


In [57]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

pipeline_lst = [
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf',MultinomialNB())]

NB_accuracy = build_clf_pipeline(all_df,pipeline_lst)

Classifier reaches accuracy score: 0.65


In [58]:
# logistic regression
from sklearn.linear_model import LogisticRegression

pipeline_lst = [
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf',LogisticRegression())]

LR_accuracy = build_clf_pipeline(all_df,pipeline_lst)

Classifier reaches accuracy score: 0.75


/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


<h4> TODO: 使用neural network:

使用neural network可以免去特征工程

In [ ]:
# CNN (法4) -> textCNN
# RNN (法5) -> textRNN